### Begin hier
Begin met het runnen van deze blokken code.  
Deze zijn verantwoordelijk voor imports, setup, dependencies en andere zaken. 

In [ ]:
# Installing Dependencies
import install_deps as deps
choiche = input("Install Deps? (choose y to continue) ")
if choiche == "y":
    deps.unattended_install()
else:
    pass

In [ ]:
# Imports
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import data_generator as dgen
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.layers import Layer
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dense, Input, Concatenate, Lambda
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.debugging.set_log_device_placement(False)
print(tf.__version__)
print(len(tf.config.list_physical_devices('GPU'))>0)

print("Succes loading Imports")

In [ ]:
# Functions
def get_file_name(path):
    file_name = path[path.rfind('/') + 1:]
    # Splits out the file-type designation
    file_name, file_extension = file_name.split('.')
    return file_name
    
print("Succes loading Functions")

### Heb je al data?
Als er al trainingsdata bestaat, schrijf dan simpelweg de filename bij het betreffende blok.  
Run anders het blok waarin de data nog word gegenereerd. 

In [ ]:
"""Ik heb geen data"""
# Start here if no data has been generated.

print("Provide the path to a video file.\n"
      "  (Example: C:/Users/admin/video/video.mp4)")

vid_path = input("Provide a video path: ")
vid_name = get_file_name(vid_path)
print(f"Provided path: '{vid_path}',\nfound filename: '{vid_name}'")
print("")

generator = dgen.Generator(vid_path)
generator.generate_data()
print(f"Video has been processed and exported to {generator.processor.file_name}__trainingsData.mp4")
print(f"Data generated and exported to {generator.processor.file_name}_trainingsData.json")
print("")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}_trainingsData.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")


print("The data-table was loaded. \nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

In [ ]:
"""Ik heb al trainingsdata. """
# Start here if data has already been generated.
print("Provide the filename to a file in "
      "the generateData_OUTPUT folder.\n"
      "Example: perfect_example_trainingsData")
      
data_path = input("Filename: ")
vid_name = get_file_name(data_path)
print(f"Provided path: {data_path}")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")

print("\nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

In [ ]:
"""Bereid de ingeladen data voor"""
# Puts X and Y for the AI to train in the proper form
Xtemp = frame_data.drop(columns=['TS', 'HR', 'PWR', 'heartRateZone'])
Y = frame_data["heartRateZone"]
X = []
temp = []

# Loop trough the different rows and append them,
#   to get rid of the "list in list" problem.
for index, row in Xtemp.iterrows():
    for i in range(0,478):
        for g in range(0,3):
            temp.append(row[i][0][g])
    X.append(temp)
    temp = []
Y.to_numpy()

# Turn the number value (1,2,3,4,5,6) in to a binary value (kinda),
# EXAMPLE: [0,0,1] == 3 etc.
Y = Y.values.reshape(-1,1)
temps = np.zeros((len(Y), 6))

for i in range(0,len(Y)):
    if Y[i][0] == 0:
        temps[i][0] = 1
    elif Y[i][0] == 1:
        temps[i][1] = 1
    elif Y[i][0] == 2:
        temps[i][2] = 1
    elif Y[i][0] == 3:
        temps[i][3] = 1
    elif Y[i][0] == 4:
        temps[i][4] = 1
    elif Y[i][0] == 5:
        temps[i][5] = 1

# print(temps)

# Split the loaded and prepped data into training data and testing data.
# The testing data is used to evaluate th created model
X_train, X_test, Y_train, Y_test = train_test_split(X, temps, test_size=0.33, random_state=42)

X_train = np.array(X_train)

# print(X_train.shape)
# print(Y_train.shape)


In [ ]:
"""Maak het model voor het AI"""
# Make the model
inputTensor = Input(shape=(1434,))
layers = []

# Add all the nodes
for i in range(0, 1434, 3):
    layers.append(Lambda(lambda x: x[:,i: i+3], output_shape=((3,)))(inputTensor))
for i in range(0, 478):
    layers[i] = Dense(1)(layers[i])
coordslayer = Concatenate()(layers)

# Define all the hidden layers
hidden1 = Dense(477*2, activation='relu')(coordslayer)
hidden2 = Dense(477, activation='relu')(hidden1)
hidden3 = Dense(240, activation='relu')(hidden2)
hidden4 = Dense(100, activation='relu')(hidden3)
hidden5 = Dense(20, activation='relu')(hidden4)

# This is the final model:
outputTensor = Dense(6, activation='sigmoid')(hidden5)
model = Model(inputs=inputTensor, outputs=outputTensor)

# Show the current model:
print(model.summary())

# TODO: Misschien een "Doorgaan? (Y/N)"

# Compile the model so it can be used
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# DONT RUN - Clears the model
keras.backend.clear_session()

In [ ]:
"""Train het model"""
model.fit(X_train, Y_train, epochs=150, batch_size=100)

In [ ]:
"""Evalueer het model"""
X_test = np.array(X_test)
accuracy = model.evaluate(X_test, Y_test)
print(accuracy)

In [ ]:
model.save('models/model_0.8_0.72')

In [ ]:
model = keras.models.load_model('models/model_0.8_0.72')